In [1]:
from selenium import webdriver

driver = webdriver.Chrome(r"C:\Users\jaehy\Downloads\driver\chromedriver.exe")

C:\Users\jaehy\AppData\Local\Temp/ipykernel_3660/1756175631.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\jaehy\Downloads\driver\chromedriver.exe")


In [2]:
main_url = "https://movie.daum.net/moviedb/grade?movieId=115112"
driver.get(main_url)

# 1번 영화 리뷰 가져오기

## 교수님께서 정해주신 최대 댓글 5000개 미만이기 때문에 전체 댓글을 끝까지 긁어올 수 있도록 더보기를 클릭하도록 했다. 

### 전체 평가 글 1254개

In [3]:
import time
time.sleep(3)
while True:
    try:
        more_xpath = '//*[@id="alex-area"]/div/div/div/div[3]/div[1]/button'
        driver.find_element('xpath', more_xpath).click()
        time.sleep(3)
    except:
        break

## BeautifulSoup 이용하여 위의 Selenium작업이 수행된 페이지를 받아서 원하는 부분을 크롤링

In [133]:
from bs4 import BeautifulSoup

html = driver.page_source
cnt_neg = 0
cnt_pos = 0

soup = BeautifulSoup(html, 'html.parser')
evaluations = soup.find_all('div', {'class':'cmt_info'})
# print(evaluations)
evaluation_lists = []
for evaluation in evaluations:
    if evaluation.find('p') is not None:
        comment = evaluation.find('p').text.strip(' \n')
        point = int(evaluation.find('div').text.strip(' \n'))
        
        if point == 10: label = 'pos'
        elif point < 6: label = 'neg'
        evaluation_lists.append((comment, label))

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.99)
Stacktrace:
Backtrace:
	Ordinal0 [0x008DACD3+2075859]
	Ordinal0 [0x0086EE61+1633889]
	Ordinal0 [0x0076B7BD+571325]
	Ordinal0 [0x00752E1A+470554]
	Ordinal0 [0x007BAA0B+895499]
	Ordinal0 [0x007CAC96+961686]
	Ordinal0 [0x007B7136+880950]
	Ordinal0 [0x0078FEFD+720637]
	Ordinal0 [0x00790F3F+724799]
	GetHandleVerifier [0x00B8EED2+2769538]
	GetHandleVerifier [0x00B80D95+2711877]
	GetHandleVerifier [0x0096A03A+521194]
	GetHandleVerifier [0x00968DA0+516432]
	Ordinal0 [0x0087682C+1665068]
	Ordinal0 [0x0087B128+1683752]
	Ordinal0 [0x0087B215+1683989]
	Ordinal0 [0x00886484+1729668]
	BaseThreadInitThunk [0x7653FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77197BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77197B8E+238]


In [114]:
# print(evaluation_lists)

평가와 평점을 크롤링 해오면서 평점이 __10점인 평은 긍정 리뷰__로, __6점 미만인 평은 부정 리뷰__로 바로 나눠서 evaluation_lists에 저장되도록 했다.

확실한 긍정, 확실한 부정으로 나누기위해 위와 같은 기준으로 나눴다.

---

In [115]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Okt
okt = Okt()

In [116]:
all_comment = []
pos_comment = []
neg_comment = []

for comment in evaluation_lists:
    all_comment.append(comment[0])
    if comment[1] == 'pos':
        pos_comment.append(comment[0])
    else:
        neg_comment.append(comment[0])
        
# print(all_comment[:5])

In [117]:
len(neg_comment)

191

In [118]:
len(pos_comment)

200

---

# 3번 나이브 베이즈

### 1번에서 완성시킨 evaluation_lists를 가져와 학습시킨다.

In [119]:
from nltk.tokenize import word_tokenize
import nltk
from collections import Counter
from konlpy.tag import Okt
okt = Okt()

In [134]:
import math

class MyNaiveBayesClassifier:
    
    def __init__(self, k = 0.5, use_morph=False):
        self.k = k
        self.word_probs = []
        self.use_morph = use_morph
        
        if self.use_morph:
            from konlpy.tag import Okt
            self.okt = Okt()
            
    def load_data(self, eval_lists):
        docs = []
        labels = []
        for line in eval_lists:
            doc, label = line[0], line[1]
            docs.append(doc)
            labels.append(label)
                
        return docs[:], labels[:]
    
    def tokenize(self, sentence):
        if self.use_morph:
            pos_sent = []
    
            sent = okt.pos(sentence, norm=True, stem=True)

            for tup in sent:
                word, tag = tup[0], tup[1]
                word_tag = word + '/' + tag
                pos_sent.append(word_tag)
            sentence = ' '.join(pos_sent)
            
        return sentence.split()
    
    def count_words(self, docs, labels):
        
        count_dict = dict()
        for doc, label in zip(docs, labels):
            for word in self.tokenize(doc):
                if word not in count_dict:
                    count_dict[word] = {'pos' : 0, 'neg' : 0}
                count_dict[word][label] += 1
        print('num of words...', len(count_dict))
#         for key in count_dict.items():
#             print(key)
        return count_dict
    
    def word_prob(self, count_dict, pos_class_num, neg_class_num, k):
        
        word_prob_list = []
        
        for key in count_dict:
            pos_word_num = count_dict[key]['pos']
            neg_word_num = count_dict[key]['neg']
            
            pos_class_prob = (pos_word_num + k) / (pos_class_num + 2*k)
            neg_class_prob = (neg_word_num + k) / (neg_class_num + 2*k)
            
            tup = (key, pos_class_prob, neg_class_prob)
            word_prob_list.append(tup)
#         print(word_prob_list)
        return word_prob_list
        
    def class_prob(self, word_prob_list, test_sentence, use_unseen=False):
        
        # p(긍정|문장), p(부정|문장) 계산
        
        test_words = self.tokenize(test_sentence)
        
        sent_log_pos_class_prob, sent_log_neg_class_prob = 0.0, 0.0
        
        for word, word_pos_class_prob, word_neg_class_prob in word_prob_list:
            if word in test_words:
                sent_log_pos_class_prob = sent_log_pos_class_prob + math.log(word_pos_class_prob)
                sent_log_neg_class_prob = sent_log_neg_class_prob + math.log(word_neg_class_prob)
            else:
                if use_unseen:
                    sent_log_pos_class_prob = sent_log_pos_class_prob + math.log(1.0 - word_pos_class_prob)
                    sent_log_neg_class_prob = sent_log_neg_class_prob + math.log(1.0 - word_neg_class_prob)
        
        sent_pos_class_prob = math.exp(sent_log_pos_class_prob)
        sent_neg_class_prob = math.exp(sent_log_neg_class_prob)
        
        pos_class_prob = sent_pos_class_prob/(sent_pos_class_prob+sent_neg_class_prob)
        neg_class_prob = sent_neg_class_prob/(sent_pos_class_prob+sent_neg_class_prob)
        
        return pos_class_prob, neg_class_prob
    
    def accuracy(self, test_file_path):
        test_docs, test_labels = self.load_data(test_file_path)
        count = 0
        
        for sentence, label in zip(test_docs, test_labels):
            pos_class_prob, neg_class_prob = self.class_prob(self.word_probs, sentence, use_unseen=False)
            
            if label == 'pos':
                if pos_class_prob > neg_class_prob: count += 1
            elif label == 'neg':
                if pos_class_prob < neg_class_prob: count += 1
            
        return count / len(test_docs)
        
    
    def train(self, eval_lists):
        
        train_docs, train_labels = self.load_data(eval_lists)
        
        word_count_dict = self.count_words(train_docs, train_labels)
        
        pos_class_num = len([label for label in train_labels if label == 'pos'])
        neg_class_num = len([label for label in train_labels if label == 'neg'])
        
        self.word_probs = self.word_prob(word_count_dict, pos_class_num, neg_class_num, self.k)
        
    def classify(self, doc, use_unseen=False):
        
        pos_class_prob, neg_class_prob = self.class_prob(self.word_probs, doc, use_unseen)
        
        if pos_class_prob > neg_class_prob:
            print('pos', pos_class_prob)
        else:
            print('neg', neg_class_prob)

In [135]:
classifier = MyNaiveBayesClassifier()
# classifier.train(evaluation_lists)

In [136]:
# test_sent = '억지스럽다'

# if use_morph: test_sent = pos_tokenize(test_sent)
# words = word_tokenize(test_sent)
# print(words)
# test_feature = {set_word: (set_word in words) for set_word in all_words}
# # print(test_feature)

classifier.classify('처음부터너무진부하고너무지루한편집', use_unseen=False)

neg 0.5
